# General description of the data

In this document is gonna be the process for do the data scraping to tyhe sites of industrial assets in belgium. 


DataFrame (url_fiche)
        ↓
Scrape PDF links from each page
        ↓
Download PDFs
        ↓
Extract text from PDFs
        ↓
Translate NL → EN
        ↓
LLM (Ollama) extraction & evaluation
        ↓
Structured assets dataset (JSON / DataFrame)


## Install the needed libraries

In [ ]:
!pip install pdfplumber pandas tqdm argostranslate langdetect
!pip install geopandas
!pip install fiona
!pip install cloudscraper
!pip install ollama

## Import the libraries

In [ ]:
import geopandas as gpd
import fiona
import pandas as pd
import ollama
from ollama import Client
import requests
from bs4 import BeautifulSoup
import argostranslate.package
import argostranslate.translate
import requests
import json
from tqdm import tqdm
import pandas as pd
import os
from concurrent.futures import ProcessPoolExecutor, as_completed
from worker_utils import process_document_task

Note: Dowloading the Language needs for this project !

In [ ]:
argostranslate.package.update_package_index()

available_packages = argostranslate.package.get_available_packages()

languages_needed = {
    ("fr", "en"),
    ("nl", "en"),
    ("de", "en"),
}

for pkg in available_packages:
    if (pkg.from_code, pkg.to_code) in languages_needed:
        argostranslate.package.install_from_path(pkg.download())


## Treatment of the initial data found

In this part i did a treatment for have the data in a good datafrae readable way ! 

In [ ]:
# Define the file name
gpkg_file = "C:/Users/m.saenzortiz/OneDrive - ENERGY POOL/Documents/PROJET Tender Office/Tendering_stock_data/data/pf_gpbv.gpkg"

## --- Step 1: List all available layers (datasets) ---
# A GeoPackage often contains multiple datasets (layers).
# We use fiona (the library geopandas relies on) to list them.

try:
    available_layers = fiona.listlayers(gpkg_file)
    print(f"File: {gpkg_file}")
    print(f"Available layers: {available_layers}")
except fiona.errors.DriverError as e:
    # If the file is not found or is corrupted
    print(f"Error listing layers: {e}")

# If layers were found, you can proceed to read one
if available_layers:
    # --- Step 2: Read a specific layer into a GeoDataFrame ---
    
    # Let's assume you want to read the first layer found
    layer_to_read = available_layers[0]

    print(f"\nReading layer: '{layer_to_read}'...")
    
    # Read the data into a GeoDataFrame (gdf)
    gdf = gpd.read_file(gpkg_file, layer=layer_to_read)

    # Display the first few rows and the data type
    print("\nSuccessfully loaded data:")
    print(gdf.head())
    print(f"\nData type: {type(gdf)}")
    print(f"Coordinate Reference System (CRS): {gdf.crs}")

else:
    print("\nCould not find any layers to read.")

File: C:/Users/m.saenzortiz/OneDrive - ENERGY POOL/Documents/PROJET Tender Office/Tendering_stock_data/data/pf_gpbv.gpkg
Available layers: ['pf_gpbv', 'layer_styles']

Reading layer: 'pf_gpbv'...

Successfully loaded data:
   id                        nummer  \
0   1  BE.VL.000000002.INSTALLATION   
1   2  BE.VL.000000004.INSTALLATION   
2   3  BE.VL.000000006.INSTALLATION   
3   4  BE.VL.000000007.INSTALLATION   
4   5  BE.VL.000000008.INSTALLATION   

                                                naam       type   gemeente  \
0                               AGC Glass Europe Mol  industrie        Mol   
1                             Agfa Gevaert_Gevaert 1  industrie    Mortsel   
2                              Agfa Gevaert_Gevaert5  industrie    Mortsel   
3  Air Liquide Large Industry_Syngas (blokveld E6...  industrie  Antwerpen   
4  Air Liquide Large Industry_Jupiter (blokveld G...  industrie  Antwerpen   

  postcode   straatnaam huisnummer busnummer vlarem_rubriek_nummer  \
0  

c:\Users\m.saenzortiz\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on 'C:/Users/m.saenzortiz/OneDrive - ENERGY POOL/Documents/PROJET Tender Office/Tendering_stock_data/data/pf_gpbv.gpkg'
  return ogr_read(


In [2]:
gdf

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,vlarem_rubriek_omschrijving,rie_rubriek_nummer,url_fiche,geometry
0,1,BE.VL.000000002.INSTALLATION,AGC Glass Europe Mol,industrie,Mol,2400,Voortstraat,27,None,20.3.4.1°b),20.3.4.1°b) - meer dan 20 ton,3.3.,https://gpbv.omgeving.vlaanderen.be/gpbv-regis...,POINT (205620 209625)
1,2,BE.VL.000000004.INSTALLATION,Agfa Gevaert_Gevaert 1,industrie,Mortsel,2640,Septestraat,27,None,7.11.2°d),7.11.2°d) - van zouten,4.2.d),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (155100.25 207532.25)
2,3,BE.VL.000000006.INSTALLATION,Agfa Gevaert_Gevaert5,industrie,Mortsel,2640,Septestraat,27,None,7.11.1°h),7.11.1°h) - kunststofmaterialen,4.1.h),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (155188 207854)
3,4,BE.VL.000000007.INSTALLATION,Air Liquide Large Industry_Syngas (blokveld E6...,industrie,Antwerpen,2040,Scheldelaan,600,None,7.11.2°a),7.11.2°a) - van gassen,4.2.a),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (142907.25 228088.5)
4,5,BE.VL.000000008.INSTALLATION,Air Liquide Large Industry_Jupiter (blokveld G...,industrie,Antwerpen,2040,Scheldelaan,600,None,7.11.2°a),7.11.2°a) - van gassen,4.2.a),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (142738 228494)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,1956,BE.VL.000002398.INSTALLATION,Verhee Luc,veeteelt,Heuvelland,8950,Nieuwkerkestraat(Nieuwk),86,None,9.3.1.d),9.3.1.d) - intensieve pluimveehouderij met mee...,6.6.a),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (42426.773 161415.773)
1956,1957,BE.VL.000002399.INSTALLATION,BoVaKip,veeteelt,Zulte,9870,Ponteputstraat,4,None,9.3.1.d),9.3.1.d) - intensieve pluimveehouderij met mee...,6.6.a),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (87716.95 180753.3)
1957,1958,BE.VL.000002400.INSTALLATION,Vanexem Geert,veeteelt,Vleteren,8640,Koekuitstraat(West),4,None,9.5.d)1°,9.5.d)1° - een intensieve pluimvee- of varkens...,6.6.a),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (35057.3 177059.4)
1958,1959,BE.VL.000002401.INSTALLATION,Dekervel Bart en Dekervel Aaron,veeteelt,Poperinge,8972,Couthoflaan,13,None,9.3.1.d),9.3.1.d) - intensieve pluimveehouderij met mee...,6.6.a),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (29486 175733)


Note: the first extract of the data for easy read and understanding!

In [3]:
gdf.to_excel("gpbv_data_v1.xlsx", index=False)

## Geting tha PDFs url from the differents sites

In [ ]:
def get_pdf_links(url):
    r = requests.get(url, timeout=15)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    pdf_links = []

    for a in soup.select("div.jumbotron a[href]"):
        link_text = a.get_text(strip=True).lower()
        href = a["href"]

        if link_text.endswith(".pdf"):
            pdf_links.append(href)

    return list(set(pdf_links)) 


In [ ]:
gdf["pdf_links"] = gdf["url_fiche"].apply(get_pdf_links)
gdf = gdf.explode("pdf_links")
gdf

Note: Second extract of data

In [ ]:
gdf.to_excel("gpbv_pdf_links.xlsx", index=False)

## Download the PDFs

In [ ]:

PDF_DIR = "pdfs"
os.makedirs(PDF_DIR, exist_ok=True)

def download_pdf(url, doc_id, nummer):
    # Get last 5 characters of the URL and sanitize for filename
    url_suffix = str(url)[-5:].replace('/', '_').replace(':', '_').replace('?', '_').replace('&', '_')
    
    # Create filename: id_nummer_last5chars.pdf
    filename = f"{doc_id}_{nummer}_{url_suffix}.pdf"
    path = os.path.join(PDF_DIR, filename)

    if not os.path.exists(path):
        r = requests.get(url, timeout=30, headers={"User-Agent": "Mozilla/5.0"})
        r.raise_for_status()
        with open(path, "wb") as f:
            f.write(r.content)

    return path

# Read the Pdf information

For this part the most important is to extract the data the most accurate, for avoid the errors and misunderstanding.

In [ ]:
test_df = pd.read_excel("gpbv_pdf_links.xlsx")
test_df = test_df.sample(3)

test_df['pdf_path'] = test_df.apply(lambda row: download_pdf(row['pdf_links'], row['id'], row['nummer']), axis=1)
test_df

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,vlarem_rubriek_omschrijving,rie_rubriek_nummer,url_fiche,geometry,pdf_links,pdf_path
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,NaN,7.11.1°b),7.11.1°b) - zuurstofhoudende koolwaterstoffen,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf
2328,124,BE.VL.000000149.INSTALLATION,Aurubis Beerse,industrie,Beerse,2340,Nieuwe dreef,33,NaN,20.2.5.,20.2.5. - De productie van ruwe non-ferrometal...,2.5.a),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (181393.72 223628.54),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\124_BE.VL.000000149.INSTALLATION_8c666.pdf
968,88,BE.VL.000000109.INSTALLATION,Evonik Antwerpen - ACA,industrie,Antwerpen,2040,Tijsmanstunnel-West,/,NaN,7.11.1°b),7.11.1°b) - zuurstofhoudende koolwaterstoffen,4.1.b),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (145952 222144),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\88_BE.VL.000000109.INSTALLATION_bc330.pdf


## Extract the information of the PDFs

In this point we extract the information of the PDFs do the traduction to english and stock the data on a txt file for an easier somprehension of the LLM model

In [ ]:
# ================== CONFIG ==================

PDF_DIR = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Scraping-PDFs-assets\PDFs"
TXT_DIR = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Scraping-PDFs-assets\TXT"
URL_BASE = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Scraping-PDFs-assets\TXT"

MAX_WORKERS = 3  # safe for laptop

# ================== HELPERS ==================

def txt_already_exists(pdf_path: str, txt_dir: str) -> bool:
    """
    Check if the TXT corresponding to a PDF already exists.
    PDF name: document.pdf -> TXT name: document.txt
    """
    pdf_filename = os.path.basename(pdf_path)
    base_name = os.path.splitext(pdf_filename)[0]
    txt_path = os.path.join(txt_dir, f"{base_name}.txt")
    return os.path.exists(txt_path)

# ================== MAIN ==================

if __name__ == "__main__":

    # -------- 1. PREPARE ITEMS (WITH RESUME SUPPORT) --------

    items_to_process = []
    already_processed = 0

    for idx, row in test_df.iterrows():
        pdf_path = row["pdf_path"]

        # ⬅️ KEY OPTIMIZATION: skip already processed PDFs
        if txt_already_exists(pdf_path, TXT_DIR):
            already_processed += 1
            continue

        item = row.to_dict()
        item["idx"] = idx
        items_to_process.append(item)

    print("=================================================")
    print(f"📄 Total PDFs in dataframe: {len(test_df)}")
    print(f"♻️ Already processed (TXT exists): {already_processed}")
    print(f"🚀 PDFs to process now: {len(items_to_process)}")
    print("=================================================\n")

    if not items_to_process:
        print("✅ Nothing to do — all PDFs already processed.")
        exit(0)

    # -------- 2. PROCESS IN PARALLEL --------

    print(f"🚀 Starting PDF → TXT extraction with {MAX_WORKERS} workers...\n")

    with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {
            executor.submit(
                process_document_task,
                item,
                PDF_DIR,
                TXT_DIR,
                URL_BASE
            ): item
            for item in items_to_process
        }

        for future in tqdm(
            as_completed(futures),
            total=len(futures),
            desc="Processing PDFs"
        ):
            try:
                idx, result = future.result()

                # Update dataframe only if processing succeeded
                if idx is not None and result:
                    test_df.at[idx, "txt_link"] = result

            except Exception as e:
                # Hard failure in worker (should be rare)
                print(f"⚠️ Worker failure: {e}")

    print("\n" + "=" * 60)
    print("✅ PDF → TXT processing finished")
    print(f"📄 Total PDFs: {len(test_df)}")
    print(f"♻️ Previously processed: {already_processed}")
    print(f"🆕 Newly processed this run: {len(items_to_process)}")
    print("=" * 60)

📄 Total PDFs in dataframe: 3
♻️ Already processed (TXT exists): 0
🚀 PDFs to process now: 3

🚀 Starting PDF → TXT extraction with 3 workers...



Processing PDFs: 100%|██████████| 3/3 [03:10<00:00, 63.42s/it] 



✅ PDF → TXT processing finished
📄 Total PDFs: 3
♻️ Previously processed: 0
🆕 Newly processed this run: 3


In [6]:
test_df

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,vlarem_rubriek_omschrijving,rie_rubriek_nummer,url_fiche,geometry,pdf_links,pdf_path,txt_sharepoint_link
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,NaN,7.11.1°b),7.11.1°b) - zuurstofhoudende koolwaterstoffen,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...
2328,124,BE.VL.000000149.INSTALLATION,Aurubis Beerse,industrie,Beerse,2340,Nieuwe dreef,33,NaN,20.2.5.,20.2.5. - De productie van ruwe non-ferrometal...,2.5.a),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (181393.72 223628.54),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\124_BE.VL.000000149.INSTALLATION_8c666.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...
968,88,BE.VL.000000109.INSTALLATION,Evonik Antwerpen - ACA,industrie,Antwerpen,2040,Tijsmanstunnel-West,/,NaN,7.11.1°b),7.11.1°b) - zuurstofhoudende koolwaterstoffen,4.1.b),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (145952 222144),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\88_BE.VL.000000109.INSTALLATION_bc330.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...


# Test if ollama is working ! 

This is the test part for ollama, for better results we can go to the site of ollama and download a more powerfull LLM model for better results if we do the proccess or stock the model in a cloud or a powerfull local machine!

https://ollama.com/library?sort=popular


In [ ]:
response = ollama.chat(
    model="llama3.1:8b",
    messages=[{"role": "user", "content": "Hello"}]
)

print(response["message"]["content"])


In [ ]:

ollama.chat(
    model="llama3.1:8b",
    messages=[{"role": "user", "content": "test"}]
)


# Implemenation of the LLM model analysis

For this proccess the flow is: 

graph TD
    A[ Read TXT File] --> B{ Asset <br/>Keywords Found?}
    
    B -- "No" --> C([End / Skip File])
    B -- "Yes" --> D[LLM Prompting & <br/>Extraction Analysis]
    
    D --> E[Generate Structured <br/>JSON Output]
    E --> F[Flatten JSON <br/>'Explode' to Rows]
    F --> G([Final Normalized <br/>Dataset])

    %% Apply Styles
    class A,D,E,F process;
    class B logic;
    class C,G startEnd;

In [ ]:
# ================== CONFIG ==================

MODEL = "llama3.1:8b"

INPUT_TXT_DIR = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Scraping-PDFs-assets\TXT"
OUTPUT_JSON_DIR = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Scraping-PDFs-assets\JSON"


In [ ]:
client = Client()


def extract_assets_with_llm(text: str):
    prompt = f"""

You are a strict Industrial Energy Auditor. 
Your task: Extract physical assets from the provided TEXT only. 

RULES:\n"

1. DO NOT use outside knowledge.\n
2. DO NOT provide URLs, links, or image paths.\n
3. If no assets are found, return an empty list [].\n
4. if not asset_type found, do not include the entry.\n
4. Return ONLY valid JSON.

Fields:
- asset_type
- capacity_value
- capacity_unit
- count_of_units

TEXT:
{text}
"""
    response = client.chat(
        model=MODEL,
        format="json",
        messages=[{"role": "user", "content": prompt}]
    )
    return json.loads(response["message"]["content"])



In [ ]:

os.makedirs(OUTPUT_JSON_DIR, exist_ok=True)

# ================== YOUR ASSET KEYWORDS (SAVED) ==================

ASSET_KEYWORDS = [
    "generator", "genset", "diesel generator", "gas generator",
    "emergency generator", "backup generator",
    "turbine", "gas turbine", "steam turbine",
    "engine", "internal combustion engine", "furnace", 

    "transformer", "power transformer", "distribution transformer",
    "substation", "switchgear", "circuit breaker",
    "electrical panel", "electrical installation",

    "battery", "batteries","battery system", "battery storage",
    "energy storage", "bess",
    "ups", "uninterruptible power supply",

    "kw", "mw", "kva", "mva", "kwh", "mwh", "vah",

    "boiler", "steam boiler", "hot water boiler",
    "heater", "furnace", "kiln", "burner",

    "chiller", "cooling system", "cooling unit",
    "cooling tower",
    "heat pump", "hvac", "air conditioning",

    "hydrogen", "h2",
    "electrolyser", "electrolyzer", "reformer",
    "natural gas", "gas installation",
    "nm3/h", "m3/h", "diesel engine", 

    "compressor", "air compressor",
    "pump", "industrial pump",
    "motor", "electric motor",

    "load shedding", "peak shaving",
    "demand response", "flexibility",
    "energy management system", "ems",

    "capacity", "rated power",
    "nominal power", "maximum power",
]


# ================== FUNCTIONS ==================

def extract_relevant_lines(text: str, max_lines: int = 5000) -> str:
    """
    Finds keyword and grabs current + next line for context.
    """
    lines = text.splitlines()
    relevant_indices = set()

    for i, line in enumerate(lines):
        if any(k in line.lower() for k in ASSET_KEYWORDS):
            relevant_indices.add(i)
            if i + 1 < len(lines):
                relevant_indices.add(i + 1) # Grab the next line too

    sorted_indices = sorted(list(relevant_indices))
    relevant = [f"[L.{idx+1:04}] {lines[idx]}" for idx in sorted_indices]
    return "\n".join(relevant[:max_lines])



def process_file(file_path: str) -> bool:
    filename = os.path.basename(file_path)
    
    try:
        with open(file_path, encoding="utf-8", errors="ignore") as f:
            text = f.read()
            
        # Quick Filter
        if not any(k in text.lower() for k in ASSET_KEYWORDS):
            return False

        relevant_text = extract_relevant_lines(text)
        if not relevant_text.strip():
            return False

        assets = extract_assets_with_llm(relevant_text)

        output_path = os.path.join(OUTPUT_JSON_DIR, filename.replace(".txt", ".json"))
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump({"source": filename, "assets": assets}, f, indent=2)

        return True
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return False

# ================== MAIN ==================

if __name__ == "__main__":
    # 1. Gather all files
    all_files = [f for f in os.listdir(INPUT_TXT_DIR) if f.endswith(".txt")]
    
    # 2. Check status
    to_process = []
    already_done_count = 0
    
    for f in all_files:
        json_filename = f.replace(".txt", ".json")
        if os.path.exists(os.path.join(OUTPUT_JSON_DIR, json_filename)):
            already_done_count += 1
        else:
            to_process.append(f)

    print("\n" + "=" * 50)
    print(f"📊 REPORT")
    print(f"📁 Total Files in folder:    {len(all_files)}")
    print(f"✅ Already Processed:        {already_done_count}")
    print(f"🚀 New Files to process:     {len(to_process)}")
    print("=" * 50 + "\n")

    if not to_process:
        print("No new files found. Everything is up to date!")
    else:
        # 3. Process remaining
        newly_processed = 0
        skipped = 0

        for f in tqdm(to_process, desc="Processing new documents"):
            ok = process_file(os.path.join(INPUT_TXT_DIR, f))
            if ok:
                newly_processed += 1
            else:
                skipped += 1

        print(f"\nProcessing Complete!")
        print(f"Newly Created: {newly_processed}")
        print(f"Skipped (no assets): {skipped}")


📊 REPORT
📁 Total Files in folder:    3
✅ Already Processed:        3
🚀 New Files to process:     0

No new files found. Everything is up to date!


In [8]:
test_df

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,vlarem_rubriek_omschrijving,rie_rubriek_nummer,url_fiche,geometry,pdf_links,pdf_path,txt_sharepoint_link
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,NaN,7.11.1°b),7.11.1°b) - zuurstofhoudende koolwaterstoffen,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...
2328,124,BE.VL.000000149.INSTALLATION,Aurubis Beerse,industrie,Beerse,2340,Nieuwe dreef,33,NaN,20.2.5.,20.2.5. - De productie van ruwe non-ferrometal...,2.5.a),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (181393.72 223628.54),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\124_BE.VL.000000149.INSTALLATION_8c666.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...
968,88,BE.VL.000000109.INSTALLATION,Evonik Antwerpen - ACA,industrie,Antwerpen,2040,Tijsmanstunnel-West,/,NaN,7.11.1°b),7.11.1°b) - zuurstofhoudende koolwaterstoffen,4.1.b),https://gpbv.omgeving.vlaanderen.be//installat...,POINT (145952 222144),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\88_BE.VL.000000109.INSTALLATION_bc330.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...


# .JSON extract data for the assets and analysis

In [ ]:
json_folder = r"C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\Documents\PROJET Assets analysis BEL\Scraping-PDFs-assets\JSON"

In [ ]:
def extract_json_data(txt_path):
    # 1. Get the filename without extension (e.g., 1117_BE...f5f1a)
    file_base = os.path.splitext(os.path.basename(txt_path))[0]
    
    # 2. Construct the path to the JSON folder
    # Adjust 'JSON_FOLDER_PATH' to your actual JSON directory
    
    json_path = os.path.join(json_folder, file_base + ".json")
    
    # 3. Read the JSON file
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
            # Return the list of assets
            return data.get("assets", {}).get("assets", [])
    return []

# Apply the function to create a new column with the list of assets
test_df['assets_list'] = test_df['txt_link'].apply(extract_json_data)

# 4. "Explode" the list into multiple rows
df_expanded = test_df.explode('assets_list')

# 5. Expand the dictionary into individual columns
# This turns {'asset_type': 'X', ...} into separate columns
assets_df = df_expanded['assets_list'].apply(pd.Series)

# Combine with the original dataframe (dropping the helper column)
final_df = pd.concat([df_expanded.drop('assets_list', axis=1), assets_df], axis=1)

# Clean up: Replace NaN or empty strings if necessary
final_df = final_df.fillna("")

In [14]:
final_df

,id,nummer,naam,type,gemeente,postcode,straatnaam,huisnummer,busnummer,vlarem_rubriek_nummer,...,rie_rubriek_nummer,url_fiche,geometry,pdf_links,pdf_path,txt_sharepoint_link,asset_type,capacity_value,capacity_unit,count_of_units
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,,7.11.1°b),...,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,boilers,7.489 litres,,
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,,7.11.1°b),...,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,boilers,"8,500 litres",,
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,,7.11.1°b),...,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,incener,"8,400 litres",,
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,,7.11.1°b),...,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,combustion plant,229 kW,,
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,,7.11.1°b),...,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,hot air generators,407 kW,,
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,,7.11.1°b),...,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,hot air generators,297 kW,,
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,,7.11.1°b),...,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,combustion plant,"2,605 kW",,
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,,7.11.1°b),...,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saenzortiz\OneDrive - ENERGY POOL\D...,boilers,"3,907 kW",,
10466,1551,BE.VL.000001903.INSTALLATION,EOC Belgium Oudenaarde - Emulsion and Compound...,industrie,Oudenaarde,9700,"Industriepark ""De Bruwaan""",24,,7.11.1°b),...,4.1.b),https://gpbv.omgeving.vlaanderen.be/installati...,POINT (95904.767 173300.233),https://gpbv.omgeving.vlaanderen.be//api/view/...,pdfs\1551_BE.VL.000001903.INSTALLATION_9441a.pdf,C:\Users\m.saen